The purpose of this notebook is to demonstrate how to apply Numba and make a small change in QMCPy codebase, achieving a signifcant reduction of run time for the example of pricing an Asian call option with multi-level Monte Carlo method in QMCPy. The main change we have made is to replace line 122 in _integrand.py with the following:

Author: Sou-Cheng Choi and Josh Herman

Date upodated: Mar 27, 2022

Date created: Mar 27, 2022

In [1]:
import os
import cProfile

In [2]:
from qmcpy import *

In [3]:
from workouts.integration_examples.asian_option_multi_level import asian_option_multi_level

## Using time or timeit magic

`%timeit`: line magic 

`%%timeit`: cell magic 

`%time`: line magic 

In [4]:
%timeit asian_option_multi_level(abs_tol=.01)


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

MeanVarData (AccumulateData Object)
    solution        5.840
    error_bound     0.010
    n_total         6822642
    n               [5349996. 1217163.  252411.]
    levels          3
    time_integrate  7.015
CubMCCLT (StoppingCriterion Object)
    abs_tol         0.010
    rel_tol         0
    n_init          2^(10)
    n_max           10000000000
    inflate         1.200
    alpha           0.010
AsianOption (Integrand Object)
    volatility      2^(-1)
    call_put        call
    start_price     30
    strike_price    25
    interest_rate   0.010
    mean_type       geometric
    multilevel_dims [ 4 16 64]
BrownianMotion (TrueMeasure Object)
    time_vec        1
    drift           0
    mean            0
    covariance      1
    decomp_type     PCA
IIDStdUniform (DiscreteDistribution Object)
    d               1
    entropy         7
    spawn_key       ()
~~~~~~~~~~~~~~

In [5]:
%time asian_option_multi_level(abs_tol=.01)


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

MeanVarData (AccumulateData Object)
    solution        5.840
    error_bound     0.010
    n_total         6645408
    n               [5564705.  890573.  187058.]
    levels          3
    time_integrate  5.609
CubMCCLT (StoppingCriterion Object)
    abs_tol         0.010
    rel_tol         0
    n_init          2^(10)
    n_max           10000000000
    inflate         1.200
    alpha           0.010
AsianOption (Integrand Object)
    volatility      2^(-1)
    call_put        call
    start_price     30
    strike_price    25
    interest_rate   0.010
    mean_type       geometric
    multilevel_dims [ 4 16 64]
BrownianMotion (TrueMeasure Object)
    time_vec        1
    drift           0
    mean            0
    covariance      1
    decomp_type     PCA
IIDStdUniform (DiscreteDistribution Object)
    d               1
    entropy         7
    spawn_key       ()
~~~~~~~~~~~~~~

## Using cprofile

The following prints long outputs:

The following command with -o captures cProfile output in a .prof file, but does not require the decorator `@profile` to be added in codebase.

In [6]:
!python -m cProfile -o ../profile/asian_option_multi_level.prof  ../workouts/integration_examples/asian_option_multi_level.py 


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

MeanVarData (AccumulateData Object)
    solution        5.841
    error_bound     0.010
    n_total         6847926
    n               [5335692. 1254388.  254774.]
    levels          3
    time_integrate  7.461
CubMCCLT (StoppingCriterion Object)
    abs_tol         0.010
    rel_tol         0
    n_init          2^(10)
    n_max           10000000000
    inflate         1.200
    alpha           0.010
AsianOption (Integrand Object)
    volatility      2^(-1)
    call_put        call
    start_price     30
    strike_price    25
    interest_rate   0.010
    mean_type       geometric
    multilevel_dims [ 4 16 64]
BrownianMotion (TrueMeasure Object)
    time_vec        1
    drift           0
    mean            0
    covariance      1
    decomp_type     PCA
IIDStdUniform (DiscreteDistribution Object)
    d               1
    entropy         7
    spawn_key       ()
~~~~~~~~~~~~~~

BTW, we have added to `makefile` a target `profile` for profiling all QMCPy workouts and tests.

The following is a magic command that gives same outputs in a pop-up window.

## Snakeviz (https://jiffyclub.github.io/snakeviz/)

The following generates an HTML page in a new browser tab and it is very readable:
http://127.0.0.1:8080/snakeviz/%2FUsers%2Fterrya%2FDocuments%2FProgramData%2FQMCSoftware%2Fworkouts%2Fintegration_examples%2Fout%2Fasian_option_multi_level.prof

To exit the following command, press stop button.

A screenshot:

In [7]:
from IPython.display import Image
Image(url= "../profile/snakeviz_profile_jit_aoml.jpg", width=1000, height=1000)

An alternative to use snakeviz with outputs captured in the Notebook:

In [8]:
%load_ext snakeviz

In [9]:
%%snakeviz
asian_option_multi_level(abs_tol=.01)


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

MeanVarData (AccumulateData Object)
    solution        5.841
    error_bound     0.010
    n_total         6638769
    n               [5589698.  862287.  183712.]
    levels          3
    time_integrate  6.772
CubMCCLT (StoppingCriterion Object)
    abs_tol         0.010
    rel_tol         0
    n_init          2^(10)
    n_max           10000000000
    inflate         1.200
    alpha           0.010
AsianOption (Integrand Object)
    volatility      2^(-1)
    call_put        call
    start_price     30
    strike_price    25
    interest_rate   0.010
    mean_type       geometric
    multilevel_dims [ 4 16 64]
BrownianMotion (TrueMeasure Object)
    time_vec        1
    drift           0
    mean            0
    covariance      1
    decomp_type     PCA
IIDStdUniform (DiscreteDistribution Object)
    d               1
    entropy         7
    spawn_key       ()
~~~~~~~~~~~~~~

Exception ignored in: <function Popen.__del__ at 0x7f95800211e0>
Traceback (most recent call last):
  File "/Users/terrya/opt/anaconda3/envs/qmcpy/lib/python3.7/subprocess.py", line 839, in __del__
    ResourceWarning, source=self)
Exception ignored in: <_io.FileIO name=71 mode='rb' closefd=True>


## Tuna

An alternative to Snakeviz for visualizing cprofile outputs

In [10]:
import tuna

The following gives a visual report in a separate tab and user need to "stop" the execution before any more notebook cells will be executed:

A screenshot:

In [11]:
from IPython.display import Image
Image(url= "../profile/tuna_profile_jit_aoml.jpg", width=1000, height=1000)

We prefer magic command instead, but `%tuna` gave blank output.

## Line-by-line profiling of time

First install `line_profiler` 3.4.0 (`https://github.com/rkern/line_profiler`) 

Need to first put `@profile` before each function that we want to do line or memory profiling.

The following two commands would need `@decorator` written to codebase.

We prefer giving the function name to be profiled to `%lprun`:

In [12]:
%load_ext line_profiler

A way to profile multiple functions in a single command is as follows. 

In [13]:
%lprun -f asian_option_multi_level -f CubMCCLT.integrate -f accumulate_data.mean_var_data.MeanVarData.update_data  -f Integrand.f asian_option_multi_level(abs_tol=.01) 


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

MeanVarData (AccumulateData Object)
    solution        5.841
    error_bound     0.010
    n_total         6637063
    n               [5595366.  853629.  184996.]
    levels          3
    time_integrate  7.414
CubMCCLT (StoppingCriterion Object)
    abs_tol         0.010
    rel_tol         0
    n_init          2^(10)
    n_max           10000000000
    inflate         1.200
    alpha           0.010
AsianOption (Integrand Object)
    volatility      2^(-1)
    call_put        call
    start_price     30
    strike_price    25
    interest_rate   0.010
    mean_type       geometric
    multilevel_dims [ 4 16 64]
BrownianMotion (TrueMeasure Object)
    time_vec        1
    drift           0
    mean            0
    covariance      1
    decomp_type     PCA
IIDStdUniform (DiscreteDistribution Object)
    d               1
    entropy         7
    spawn_key       ()
~~~~~~~~~~~~~~

## Using memory_profiler

Install `memory_profiler` 0.60.0 from `https://pypi.org/project/memory-profiler/`

In [14]:
%load_ext memory_profiler

`%memit`: line magic similar to `%timeit`
`%%memit`: cell magic like `%%timeit`

In [15]:
%memit asian_option_multi_level(abs_tol=.01)


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

MeanVarData (AccumulateData Object)
    solution        5.841
    error_bound     0.010
    n_total         6652328
    n               [5545333.  914642.  189281.]
    levels          3
    time_integrate  10.121
CubMCCLT (StoppingCriterion Object)
    abs_tol         0.010
    rel_tol         0
    n_init          2^(10)
    n_max           10000000000
    inflate         1.200
    alpha           0.010
AsianOption (Integrand Object)
    volatility      2^(-1)
    call_put        call
    start_price     30
    strike_price    25
    interest_rate   0.010
    mean_type       geometric
    multilevel_dims [ 4 16 64]
BrownianMotion (TrueMeasure Object)
    time_vec        1
    drift           0
    mean            0
    covariance      1
    decomp_type     PCA
IIDStdUniform (DiscreteDistribution Object)
    d               1
    entropy         7
    spawn_key       ()
~~~~~~~~~~~~~

Note that MiB is megibytes. See, for example, `https://www.majordifferences.com/2018/03/differences-between-megabyte-and.html`.

The following needs @profile in the function to be profiled

The following tries to save output

We can chain multiple functions to perform memory profiling:

In [16]:
%mprun -f asian_option_multi_level -f CubMCCLT.integrate -f accumulate_data.mean_var_data.MeanVarData.update_data -f Integrand.f asian_option_multi_level(abs_tol=.01) 


~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

MeanVarData (AccumulateData Object)
    solution        5.841
    error_bound     0.010
    n_total         6630049
    n               [5677237.  763677.  186063.]
    levels          3
    time_integrate  9.562
CubMCCLT (StoppingCriterion Object)
    abs_tol         0.010
    rel_tol         0
    n_init          2^(10)
    n_max           10000000000
    inflate         1.200
    alpha           0.010
AsianOption (Integrand Object)
    volatility      2^(-1)
    call_put        call
    start_price     30
    strike_price    25
    interest_rate   0.010
    mean_type       geometric
    multilevel_dims [ 4 16 64]
BrownianMotion (TrueMeasure Object)
    time_vec        1
    drift           0
    mean            0
    covariance      1
    decomp_type     PCA
IIDStdUniform (DiscreteDistribution Object)
    d               1
    entropy         7
    spawn_key       ()
~~~~~~~~~~~~~~